In [1]:
pip install google_play_scraper

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-0.1.1-cp36-none-any.whl size=22260 sha256=fd9d02ea402747a6f074e631e024a3d9b7da14692daa90b212bcb0bd7e4f388d
  Stored in directory: /root/.cache/pip/wheels/d7/1f/71/e2b30aab85297ad6dd2e3049587a6763cfb7e803a0b76d982e
Successfully built google-play-scraper


In [2]:
import os

# verifying and removing the file, if exists from google drive

if os.path.exists("app_review.csv"):
  os.remove("app_review.csv")
  print("The file is deleted.")
else:
  print("The file does not exist.")

The file does not exist.


In [3]:
import multiprocessing
from google_play_scraper import Sort, reviews
import re
import sys


def web_scrapping(x):
  for i in range (x,x+3):
    try:
      result, continuation_token = reviews(
        'com.google.android.apps.youtube.kids',
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=Sort.RATING, # None, defaults to Sort.MOST_RELEVANT
        count=i, # defaults to 100
        filter_score_with=5
      )

      # If we pass 'continuation_token' as an argument to the reviews function at this point,
      # it will crawl the items after "i-th" review item.
          
      result, _ = reviews(
        'com.google.android.apps.youtube.kids',
        continuation_token=continuation_token # defaults to None(load from the beginning)
      )

      # remove unicode characters from data
      txt_extract = str(result)
      encoded_string = txt_extract.encode("ascii", "ignore")
      decode_string = encoded_string.decode()
      txt = decode_string
        
      # pre-cleaning of each review before storing into the file app_review.csv
      txt = re.sub("'","",txt)
      txt = re.sub("\"","",txt)
      txt = re.sub("#","",txt)
        
      # extracting username
      user = txt.split('userName: ')[1].split(', userImage')[0]
        
      # extracting rating
      rating = txt.split('score: ')[1].split(', thumbsUpCount')[0]
        
      # extracting likes
      upvote = txt.split('thumbsUpCount: ')[1].split(', reviewCreatedVersion')[0]
        
      # breaking the datetime in each review to year, month, day, hour, minute, second
      datetime = re.search('datetime.datetime\((.+?)\),', txt).group(1)
      datetime = re.sub(", ",",",datetime)
        
      # extracting review
      review = re.search('content: (.+?), score', txt).group(1)
      review = re.sub("!","",review)
      review = re.sub(",","",review)
      review = "'"+review+"'"

      print("i : ",i)
      print(user)
      print(datetime)
      print(rating)
      print(upvote)
      print(review)

      # created and opened file in append mode
      file = open('app_review.csv','a')
      file.write(user+', '+rating+', '+upvote+', '+review+', '+datetime+'\n')

    finally:
          file.close()
  return

# applying multi-processing to extract reviews 5x faster - p1, p2, p3, p4, p5
# each child process will extract 100 reviews
# we can add as many child processes as we like.

i=1
p1 = multiprocessing.Process(target=web_scrapping,args=[i])
p2 = multiprocessing.Process(target=web_scrapping,args=[i+100])
p3 = multiprocessing.Process(target=web_scrapping,args=[i+200])
p4 = multiprocessing.Process(target=web_scrapping,args=[i+300])
p5 = multiprocessing.Process(target=web_scrapping,args=[i+400])


# starting the child processes

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()


# joining the child processes to parent process

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()


i :  1
Yusuf Patel
2019,1,29,10,5
5
0
'nice'
i :  2
Vishakha Muley
2019,11,11,10,56,55
5
0
'Very nice app'
i :  3
Nathan Agda
2017,1,5,0,2,36
i :  101
5
A Google user
0
'Love it'
2018,7,27,17,38,44
5
2
'This app made me feel special and educational'
i :  102
Alyssa Baker
2017,12,17,17,56,38
5
19
'You kids need to watch this its not about nothing that it cant do or die I mean their children and and you can also check on your kid'
i :  201
anita agarwal
2019,10,4,14,8,48
5
0
'Nice app'
i :  103
Carl Boyd
2019,8,16,18,54,42
5
0
'best ever'
i :  202
Saurabh Saini
2020,4,22,19,49,33
5
0
'Awesome'
i :  301
Mackenzie6512
2018,3,7,1,8,42
5
1
'I love this app'
i :  203
ANIL KUMAR
2020,8,16,14,20,15
5
0
'Good'
i :  302
Dino Gilmore
2020,8,4,21,48,54
5
0
'Fantastic'
i :  401
Cierra Perry
2017,12,21,23,53,28
5
0
'I love YouTubekids I am obsessed'
i :  303
Muhammad sajjad Khan
2020,8,26,9,37,4
5
0
'Buht khoob'
i :  402
Nadine Nicolene Marais
2019,5,15,9,45,39
5
0
'Fantastic'
i :  403
Jade Fernandes

In [4]:
# downloading the .csv file into local drive

from google.colab import files
files.download('app_review.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>